In [1]:
from sshtunnel import SSHTunnelForwarder
import psycopg2
import pandas as pd
import math
from psycopg2.extras import execute_values
 
server = SSHTunnelForwarder(
    ssh_address_or_host=('10.12.0.8', 2043),
    ssh_username='root', 
    ssh_password = 'ynby20250625', 
 
    
    remote_bind_address=('localhost' , 5432)) 
server.start()
conn = psycopg2.connect(database = 'FAERS20-24', 
                            user = 'postgres',     
                            password = 'woshipostgres', 
                            host = '127.0.0.1',   
                            port = server.local_bind_port)

In [ ]:
def get_drug_reaction(drugid, con = conn):
    sql = (f"SELECT DISTINCT reactid FROM cdir_fo WHERE drugbank_id = '{drugid}';")
    drug_reaction = pd.read_sql(sql, con = conn)
    #print(drug_reaction)
    reactids = drug_reaction.iloc[:, 0]
    return reactids


def count_drug(drugid, con = conn):
    '''
    Figure out the value 'a+c'(the sum of recordings of all reactions 
    triggered by the given drug) in the Disproportional Method
    '''
    sql = (f"SELECT count FROM drugcount WHERE drugbank_id = '{drugid}';")
    drugcount = pd.read_sql(sql, con=conn)
    return drugcount['count'].values[0]


def count_react(reactid, con = conn):
    '''
    Figure out the value 'a+b'(the sum of recordings of the given reaction 
    ) in the Disproportional Method
    '''
    sql = (f"SELECT count FROM reactcount WHERE reactid = '{reactid}';")
    reactcount = pd.read_sql(sql, con=conn)
    return reactcount['count'].values[0]


def count_a(drugid, reactid, con = conn):
    '''
    Figure out the value 'a' (the count of recordings associating 
    with given drug and focusing adverse reaction) 
    in the Disproportional Method
    '''
    sql = (f"SELECT count(*) FROM cdir_fo WHERE drugbank_id = '{drugid}' AND reactid = '{reactid}';")
    reactcount = pd.read_sql(sql, con=conn)
    return reactcount['count'].values[0]


def ror_ci(a, b, c, d, ror):
    ror_upper_ci_power = math.log(ror) + 1.96 * math.sqrt(1/a + 1/b + 1/c + 1/d)
    ror_lower_ci_power = math.log(ror) - 1.96 * math.sqrt(1/a + 1/b + 1/c + 1/d)
    ror_upper_ci = math.exp(ror_upper_ci_power)
    ror_lower_ci = math.exp(ror_lower_ci_power)
    return float('%.2f' % ror_lower_ci), float('%.2f' % ror_upper_ci)


def prr_ci(a, b, c, d, prr):
    prr_upper_ci_power = math.log(prr) + 1.96 * math.sqrt(1/a + 1/b - 1/(a + c) - 1/(b + d))
    prr_lower_ci_power = math.log(prr) - 1.96 * math.sqrt(1/a + 1/b - 1/(a + c) - 1/(b + d))
    prr_upper_ci = math.exp(prr_upper_ci_power)
    prr_lower_ci = math.exp(prr_lower_ci_power)
    return float('%.2f' % prr_lower_ci), float('%.2f' % prr_upper_ci)

def insert_batch_data(cursor, data_batch):
    '''
    Insert values into table in batch
    '''
    insert_sql = """
    INSERT INTO faersmining (drugid, reactid, a, b, c, d, ror, ror_lower_ci, ror_upper_ci, prr, prr_lower_ci, prr_upper_ci)
    VALUES %s
    """
    try:
        execute_values(cursor, insert_sql, data_batch, template=None, page_size=100)
        return True
    except Exception as e:
        print(f"Error: {e}")
        return False

In [ ]:

def calculate(con = conn, batch_size = 1000):
    total = 6981059
    sql = ("SELECT DISTINCT drugbank_id FROM drug21_24_map ORDER BY drugbank_id ASC")
    drugids = pd.read_sql(sql, con = conn)


    cursor = con.cursor()
    data_batch = []
    total_inserted = 0

    try:
        for drugid in drugids['drugbank_id']:
            try:
                print(f"Processing: {drugid}")
                reactids = get_drug_reaction(drugid)
                ac = count_drug(drugid)
                for reactid in reactids:
                    ab = count_react(reactid)
                    a = count_a(drugid, reactid)
                    b = ab - a
                    c = ac - a
                    d = total - ab - ac + a
                    if b == 0 or c == 0:
                        a+=0.5
                        b+=0.5
                        c+=0.5
                        d+=0.5
                    ror = (a * d) / (b * c)
                    ror_lower_ci, ror_upper_ci = ror_ci(a, b, c, d, ror)

                    prr = a * (b + d)/(b * ac)
                    prr_lower_ci, prr_upper_ci = prr_ci(a, b, c, d, prr)

                    row_data = (
                        str(drugid), str(reactid), 
                        float(a), float(b), float(c), float(d), 
                        float('%.2f' % ror), ror_lower_ci, ror_upper_ci, 
                        float('%.2f' % prr), prr_lower_ci, prr_upper_ci
                    )
                    data_batch.append(row_data)
                    if len(data_batch) >= batch_size:
                        if insert_batch_data(cursor, data_batch):
                            con.commit()
                        else:
                            con.rollback()
                        data_batch = []


                    #print(drugid, reactid, a, b, c, d, '%.2f' % ror, ror_lower_ci, ror_upper_ci, '%.2f' % prr, prr_lower_ci, prr_upper_ci)
                    #cur = conn.cursor()
                    #cur.execute(f"INSERT INTO faersmining (drugid, reactid, a, b, c, d, ror, ror_lower_ci, ror_upper_ci, prr, prr_lower_ci, prr_upper_ci) VALUES('{drugid}', '{reactid}', {a}, {b}, {c}, {d}, {ror}, {ror_lower_ci}, {ror_upper_ci}, {prr}, {prr_lower_ci}, {prr_upper_ci})")
            except Exception as e:
                    print(f"Error When {drugid}-{reactid}: {e}")
                    continue
            
        if data_batch:
            if insert_batch_data(cursor, data_batch):
                con.commit()
            else:
                con.rollback()
    
    except Exception as e:
        print(f"Error: {e}")
        con.rollback()
    finally:
        cursor.close()        



In [11]:
calculate()

Processing:  LYSINE AMIDOTRIZOATE
Processing:  POMT 
Processing:  TOPAMAX |ZANAFLEX
Processing: (1-743)-(1638-2332)-BLOOD-COAGULATION FACTOR VIII (SYNTHETIC HUMAN) FUSION PROTEIN WITH IMMUNOGLOBULIN G1 (SYNTHETIC HUMAN FC DOMAIN FRAGMENT), (1444-6^),(1447-9^)-BIS(DISULFIDE) WITH IMMUNOGLOBULIN G1 (SYNTHETIC HUMAN FC DOMAIN FRAGMENT)
Processing: (125I) IOD HUMANALBUMIN
Processing: (6S)-5-METHYLTETRAHYDROFOLATE GLUCOSAMINE, COLECALCIFEROL, FOLIC ACID,
Processing: (6S)-5-METHYLTETRAHYDROFOLATE, ASCORBIC ACID, BETACAROTENE, BIOTIN, BO
Processing: (6S)?5?METHYLTETRAHYDROFOLATE GLUC
Processing: (6S)?5?METHYLTETRAHYDROFOLATE GLUCOSAMINE
Processing: (??)-??-PINENE
Processing: (ADVAIR DISKUS) 500-50 MCG
Processing: (ASCORBIC ACID) ELDERBERRY FRUIT
Processing: (BRENTUXIMAB VEDOTIN
Processing: (CLINOLEIC)LONG CHAIN FAT EMULSION INJECTION (OO)
Processing: (LOPERAMIDE)IMODIUM
Processing: (MINOXIDIL
Processing: (MVI GUMMY,+ GUMMY, HAIR
Processing: (NON?ABBVIE)
Processing: (NOW RITALIN 10 MG X3
Proce